In [49]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2

'''
# if cannot import the modules, add the parent directory to system path might help

import os
import sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/'
sys.path.append(parent_dir)
'''

import os
from utils.path import dir_HugeFiles
from utils.words import make_corpus_0, clean_wordcount, get_wordcount, replace_UNK, parse_section
from utils.preprocessing import load, preprocessing, clean_ny
from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv, print_time
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

dir_save = os.path.normpath(dir_HugeFiles+'dph/dic_20190607.pickle')
dic = load(dir_save)
random_seed = 2019

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.3 µs
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
exist
time: 12.4 s


In [24]:
ls = [i for i,v in dic.items() if len(v['ingredients'])>1]
print('drop %d recipes with less than 2 ingredients' %(len(dic)-len(ls)))

drop 46 recipes with less than 2 ingredients
time: 139 ms


In [25]:
ls = [i for i in ls if len(dic[i]['directions'])>1]
print('furthur drop %d recipes with less than 2 instructions' %(len(dic)-len(ls)))

furthur drop 1026 recipes with less than 2 instructions
time: 88.7 ms


In [76]:
# split data
train_ls, test_ls = train_test_split(ls, test_size = 0.2, shuffle = True, random_state = random_seed)
train_ls, val_ls = train_test_split(train_ls, test_size = 0.25, shuffle = True, random_state = random_seed + 1)

time: 82.2 ms


In [79]:
# show some examples
ingredients_list = dic[0]['ingredients_list']
to_write = ' $ '.join([' '.join(ele) for ele in ingredients_list])
print(to_write)
print()
directions_list = dic[0]['directions_list']
to_write = ' '.join([' '.join(ele) for ele in directions_list])
print(to_write)
print()

1/2 cup ketchup $ 1/4 cup prepared yellow mustard $ 1 cup cornflake crumbs $ 1 ( 16 ounce ) package all beef hot dogs

preheat oven to 350 degrees f ( 175 degrees c ) . line a baking sheet with a sheet of aluminum foil . stir together ketchup and mustard on a plate until mixed . place the cornflake crumbs in a shallow bowl . roll each hot dog in the ketchup mixture , then roll in the cornflake crumbs to coat . place onto prepared baking sheet . bake in preheated oven until the hot dogs are hot on the inside , and crispy on the outside , 15 to 20 minutes .

time: 25.6 ms


In [71]:
def to_opennmt(key, ls, filename, delimiter = ' $ ', overwrite = False):
    '''
    params: key: string, the key of data, e.g. dic[0]['ingredients_list']
    params: ls: list, he index of recipe, from train_test_split
    params: filename, string, the filename of txt file
    '''
    make_dir(filename)
    if os.path.isfile(filename) == True and overwrite == False:
        print('already exists'+filename)
    else:    
        with open(filename,'w') as f:
            for i, v in dic.items():
                if i in ls:
                    to_write = delimiter.join([' '.join(ele) for ele in v[key]])
                    f.write("%s\n" % to_write)
        print('saved '+filename)

time: 32.1 ms


In [83]:
print('size after train test split')
print([len(ls) for ls in [train_ls, val_ls, test_ls]])

size after train test split
[32445, 10815, 10816]
time: 34.2 ms


In [84]:
# takes about 3 min
to_opennmt(key = 'ingredients_list', 
           ls = train_ls, 
           filename = dir_HugeFiles+'to_opennmt/train.ingred.atok'
          )
to_opennmt(key = 'ingredients_list', 
           ls = val_ls, 
           filename = dir_HugeFiles+'to_opennmt/val.ingred.atok'
          )
to_opennmt(key = 'ingredients_list', 
           ls = test_ls, 
           filename = dir_HugeFiles+'to_opennmt/test.ingred.atok'
          )
to_opennmt(key = 'directions_list', 
           ls = train_ls, 
           filename = dir_HugeFiles+'to_opennmt/train.instr.atok',
           delimiter = ' '
          )
to_opennmt(key = 'directions_list', 
           ls = val_ls, 
           filename = dir_HugeFiles+'to_opennmt/val.instr.atok'
          )
to_opennmt(key = 'directions_list', 
           ls = test_ls, 
           filename = dir_HugeFiles+'to_opennmt/test.instr.atok'
          )

time: 2min 14s


In [88]:
corpus ="""
Monty Python (sometimes known as The Pythons) were a British surreal comedy group who created the sketch comedy show Monty Python's Flying Circus,
that first aired on the BBC on October 5, 1969. Forty-five episodes were made over four series. The Python phenomenon developed from the television series
into something larger in scope and impact, spawning touring stage shows, films, numerous albums, several books, and a stage musical.
The group's influence on comedy has been compared to The Beatles' influence on music."""
import collections, nltk
# we first tokenize the text corpus
tokens = nltk.word_tokenize(corpus)

#here you construct the unigram language model 
def unigram(tokens):    
    model = collections.defaultdict(lambda: 0.01)
    for f in tokens:
        try:
            model[f] += 1
        except KeyError:
            model [f] = 1
            continue
    N = float(sum(model.values()))
    for word in model:
        model[word] = model[word]/N
    return model

#computes perplexity of the unigram model on a testset  
def perplexity(testset, model):
    testset = testset.split()
    perplexity = 1
    N = 0
    for word in testset:
        N += 1
        perplexity = perplexity * (1/model[word])
    perplexity = pow(perplexity, 1/float(N)) 
    return perplexity

testset1 = "Monty"
testset2 = "abracadabra gobbledygook rubbish"

model = unigram(tokens)
print(perplexity(testset1, model))
print(perplexity(testset2, model))


49.09452736318415
99.99999999999997
time: 64 ms
